In [18]:
# 层和块
''' 
块（block）可以描述单个层、由多个层组成的组件或整个模型本⾝
使⽤块进⾏抽象的⼀个好处是可以将⼀些块组合成更⼤的组件，这⼀过程通常是递归的

从编程的⻆度来看，块由类（class）表⽰。它的任何⼦类都必须定义⼀个将其输⼊转换为输出的前向传播函
数，并且必须存储任何必需的参数。注意，有些块不需要任何参数。最后，为了计算梯度，块必须具有反向
传播函数。在定义我们⾃⼰的块时，由于⾃动微分（在 2.5节 中引⼊）提供了⼀些后端实现，我们只需要考
虑前向传播函数和必需的参数

在构造⾃定义块之前，我们先回顾⼀下多层感知机（4.3节 ）的代码。下⾯的代码⽣成⼀个⽹络，其中包含⼀
个具有256个单元和ReLU激活函数的全连接隐藏层，然后是⼀个具有10个隐藏单元且不带激活函数的全连接
输出层
'''

import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20) # rand 均匀分布 randn 标准正态分布
net(X)
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([256, 20])) ('0.bias', torch.Size([256])) ('2.weight', torch.Size([10, 256])) ('2.bias', torch.Size([10]))


In [19]:
''' 
在这个例⼦中，我们通过实例化nn.Sequential来构建我们的模型，层的执⾏顺序是作为参数传递的。简⽽⾔
之，nn.Sequential定义了⼀种特殊的Module，即在PyTorch中表⽰⼀个块的类，它维护了⼀个由Module组
成的有序列表。注意，两个全连接层都是Linear类的实例，Linear类本⾝就是Module的⼦类。另外，到⽬
前为⽌，我们⼀直在通过net(X)调⽤我们的模型来获得模型的输出。这实际上是net.__call__(X)的简
写。这个前向传播函数⾮常简单：它将列表中的每个块连接在⼀起，将每个块的输出作为下⼀个块的输⼊。
'''
# ⾃定义块
''' 
在下⾯的代码⽚段中，我们从零开始编写⼀个块。它包含⼀个多层感知机，其具有256个隐藏单元的隐藏层
和⼀个10维输出层。注意，下⾯的MLP类继承了表⽰块的类。我们的实现只需要提供我们⾃⼰的构造函数
（Python中的__init__函数）和前向传播函数
'''
class MLP(nn.Module): # MLP 多层感知器(Multi-Layer Perceptron，MLP) 输入 隐藏 输出
    # ⽤模型参数声明层。这⾥，我们声明两个全连接的层
    def __init__(self):
        # 调⽤MLP的⽗类Module的构造函数来执⾏必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__() # 子类构造函数调用super().init()的时候，会从父类继承属性
        self.hidden = nn.Linear(20, 256) # 输入层-隐藏层
        # 隐藏层
        self.out = nn.Linear(256, 10)    # 隐藏层- 输出层
        # 输出层
        # 定义模型的前向传播，即如何根据输⼊X返回所需的模型输出
    def forward(self, X):
        # 注意，这⾥我们使⽤ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X))) 


In [20]:
''' 
我们⾸先看⼀下前向传播函数，它以X作为输⼊，计算带有激活函数的隐藏表⽰，并输出其未规范化的输出
值。在这个MLP实现中，两个层都是实例变量。要了解这为什么是合理的，可以想象实例化两个多层感知机
（net1和net2），并根据不同的数据对它们进⾏训练。当然，我们希望它们学到两种不同的模型。

接着我们实例化多层感知机的层，然后在每次调⽤前向传播函数时调⽤这些层。注意⼀些关键细节：⾸先，我
们定制的__init__函数通过super().__init__() 调⽤⽗类的__init__函数，省去了重复编写模版代
码的痛苦。然后，我们实例化两个全连接层，分别为self.hidden和self.out。注意，除⾮我们实现⼀个
新的运算符，否则我们不必担⼼反向传播函数或参数初始化，系统将⾃动⽣成这些

我们来试⼀下这个函数:
'''
net = MLP()
net(X)
print(*[(name, param.shape) for name, param in net.named_parameters()])

('hidden.weight', torch.Size([256, 20])) ('hidden.bias', torch.Size([256])) ('out.weight', torch.Size([10, 256])) ('out.bias', torch.Size([10]))


In [21]:
''' 
块的⼀个主要优点是它的多功能性。我们可以⼦类化块以创建层（如全连接层的类）、整个模型（如上⾯
的MLP类）或具有中等复杂度的各种组件。我们在接下来的章节中充分利⽤了这种多功能性，⽐如在处理卷
积神经⽹络时
'''
# 顺序块
''' 
现在我们可以更仔细地看看Sequential类是如何⼯作的，回想⼀下Sequential的设计是为了把其他模块
串起来。为了构建我们⾃⼰的简化的MySequential，我们只需要定义两个关键函数：
1. ⼀种将块逐个追加到列表中的函数。
2. ⼀种前向传播函数，⽤于将输⼊按追加块的顺序传递给块组成的“链条”。
下⾯的MySequential类提供了与默认Sequential类相同的功能。
'''
''' 
python函数星号参数详解:

函数在定义中，通过在参数前加星号，将传递进来的多个参数转化为一个对象，元组或字典，可以说是将这些参数值收集起来

参数前加一个星号，表明将所有的值放在同一个元组中，该参数的返回值是一个元组。
参数前加两个星号，表明将所有的值放在同一个字典中，该参数的返回值是一个字典。
--------------------------------------------------------------------------------------------------
参数带星号表示支持可变不定数量的参数，这种方法叫参数收集。
星号又可以带1个或2个，带1个表示按位置来收集参数，带2个星号表示按关键字来收集参数

1、带一个星号的参数收集模式：
这种模式是在函数定义时在某个形参前面加一个星号，调用时按位置匹配不带星号的形参和实参，
多余的实参都将作为一个元组的元素保存到星号对应的形参中。

2、带两个星号的参数收集模式：
带1个星号的参数收集模式的收集参数不能收集关键字参数传递的实参，要收集关键字参数传递的实参，
需要在收集参数前使用两个星号，此时收集参数对应的是一个字典而不是元组。

'''
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__() # 继承父类的属性
        for idx, module in enumerate(args): # enumerate 枚举,列举
            # 这⾥，module是Module⼦类的⼀个实例。我们把它保存在'Module'类的成员
            # 变量_modules中,module的类型是OrderedDict (有序词典)
            self._modules[str(idx)] = module
    def forward(self, X):
            # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values(): # block 块
            X = block(X)
        return X



In [22]:
''' 
__init__函数将每个模块逐个添加到有序字典_modules中。你可能会好奇为什么每个Module都有⼀
个_modules属性？以及为什么我们使⽤它⽽不是⾃⼰定义⼀个Python列表？简⽽⾔之，_modules的主要
优点是：在模块的参数初始化过程中，系统知道在_modules字典中查找需要初始化参数的⼦块。

当MySequential的前向传播函数被调⽤时，每个添加的块都按照它们被添加的顺序执⾏。现在可以使⽤我
们的MySequential类重新实现多层感知机
'''
# 请注意，MySequential的⽤法与之前为Sequential类编写的代码相同（如 4.3节 中所述）
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([256, 20])) ('0.bias', torch.Size([256])) ('2.weight', torch.Size([10, 256])) ('2.bias', torch.Size([10]))


In [23]:
# 在前向传播函数中执⾏代码
''' 
Sequential类使模型构造变得简单，允许我们组合新的架构，⽽不必定义⾃⼰的类。然⽽，并不是所有的
架构都是简单的顺序架构。当需要更强的灵活性时，我们需要定义⾃⼰的块。例如，我们可能希望在前向传
播函数中执⾏Python的控制流。此外，我们可能希望执⾏任意的数学运算，⽽不是简单地依赖预定义的神经
⽹络层。
到⽬前为⽌，我们⽹络中的所有操作都对⽹络的激活值及⽹络的参数起作⽤。然⽽，有时我们可能希望合并
既不是上⼀层的结果也不是可更新参数的项，我们称之为常数参数（constant parameter）。例如，我们需要
⼀个计算函数 f(x, w) = c · w⊤x的层，其中x是输⼊，w是参数，c是某个在优化过程中没有更新的指定常量。
因此我们实现了⼀个FixedHiddenMLP类，如下所⽰：
'''
class FixedHiddenMLP(nn.Module): # MLP 多层感知机
    def __init__(self):
        super().__init__() # 继承父类属性
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 随机权重 rand [0,1)均匀分布
        self.linear = nn.Linear(20, 20) 
    def forward(self, X):
        X = self.linear(X)
        # 使⽤创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1) # 权重初始化后为常量
        # 复⽤全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [24]:
''' 
在这个FixedHiddenMLP模型中，我们实现了⼀个隐藏层，其权重（self.rand_weight）在实例化时被
随机初始化，之后为常量。这个权重不是⼀个模型参数，因此它永远不会被反向传播更新。然后，神经⽹络
将这个固定层的输出通过⼀个全连接层。
注意，在返回输出之前，模型做了⼀些不寻常的事情：它运⾏了⼀个while循环，在L1范数⼤于1的条件下，将
输出向量除以2，直到它满⾜条件为⽌。最后，模型返回了X中所有项的和。注意，此操作可能不会常⽤于在
任何实际任务中，我们只是向你展⽰如何将任意代码集成到神经⽹络计算的流程中。
'''
net = FixedHiddenMLP() # 自定义的隐藏层
net(X) # 网络实例化
print(*[(name, param.shape) for name, param in net.named_parameters()])

('linear.weight', torch.Size([20, 20])) ('linear.bias', torch.Size([20]))


In [27]:
# 我们可以混合搭配各种组合块的⽅法。在下⾯的例⼦中，我们以⼀些想到的⽅法嵌套块。
class NestMLP(nn.Module): 
    def __init__(self):
        super().__init__() # 继承父类属性
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU(),
                                 nn.Linear(32,32),nn.ReLU()) # 线性层 -> 激活 网络定义
        self.linear = nn.Linear(32, 16) # 定义线性层
    def forward(self, X):
        return self.linear(self.net(X)) # 前向传播 线性层 激活 线性层 激活 线性层
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP()) # 嵌套
chimera(X)
print(*[(name, param.shape) for name, param in chimera.named_parameters()])

('0.net.0.weight', torch.Size([64, 20])) ('0.net.0.bias', torch.Size([64])) ('0.net.2.weight', torch.Size([32, 64])) ('0.net.2.bias', torch.Size([32])) ('0.net.3.weight', torch.Size([32, 32])) ('0.net.3.bias', torch.Size([32])) ('0.linear.weight', torch.Size([16, 32])) ('0.linear.bias', torch.Size([16])) ('1.weight', torch.Size([20, 16])) ('1.bias', torch.Size([20])) ('2.linear.weight', torch.Size([20, 20])) ('2.linear.bias', torch.Size([20]))


In [ ]:
''' 
⼩结
• ⼀个块可以由许多层组成；⼀个块可以由许多块组成。
• 块可以包含代码。
• 块负责⼤量的内部处理，包括参数初始化和反向传播。
• 层和块的顺序连接由Sequential块处理
'''

